In [2]:
import tensorflow as tf
from decimal import * 
import random
import pandas as pd

/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/miniconda3/envs/tensorflow/lib/python3.6/site-pack

In [7]:
def createSample(group,sex):
    
    if sex== 0:
        height=random.gauss(178.4,7.6)
        ability1=random.gauss(MEANS[format(prob_men,'.2f')][0],STD_ABILITY1)
        ability2=random.gauss(MEANS[format(prob_men,'.2f')][1],STD_ABILITY2)
        ability1_noise=ability1+random.gauss(0,STD_NOISE)
        ability2_noise=ability2+random.gauss(0,STD_NOISE)
    elif sex == 1:
        height=random.gauss(164.7,7.1)
        ability1=random.gauss(MEANS[format(prob_men+diff_prob, '.2f')][0],STD_ABILITY1)
        ability2=random.gauss(MEANS[format(prob_men+diff_prob, '.2f')][1],STD_ABILITY2)
        ability1_noise=ability1+random.gauss(0,STD_NOISE)
        ability2_noise=ability2+random.gauss(0,STD_NOISE)

    if ability1+ability2>=40:
        label=1
    else:
        label=0
    
    
    sample= [group, numb_dataset, seed, ratio, prob_men, prob_women, diff_prob, ability1_noise, ability2_noise, height, sex, label]
    return sample

def cleanDataSet(data, prob, ratio, diff):
    n1=1000
    cleanData= []
    prob_w = round(prob,2) + round(diff, 2)
    ratio_w = 1 - round(ratio, 2)
    (countM_0, countM_1) = (0,0)
    (countW_0, countW_1) = (0,0)
    (stopM_0, stopM_1) = (False,False)
    (stopW_0, stopW_1) = (False,False)
    
    for row in data:
        if (row[10] == 0):
            if ((stopM_1 == False) & (row[11] == 1)):
                countM_1 += 1
                if (countM_1 <= round(n1*round(ratio,2)*round(prob,2))):
                    cleanData.append(row)
                else:
                    stopM_1 = True
                
            elif ((stopM_0 == False) & (row[11] == 0)):
                countM_0 += 1
                if (countM_0 <= round(n1*round(ratio,2)*(1-round(prob,2)))):
                    cleanData.append(row)
                else:
                    stopM_0 = True
        else:
            if ((stopW_1 == False) & (row[11] == 1)):
                countW_1 += 1
                if (countW_1 <= round(n1*prob_w*ratio_w)):
                    cleanData.append(row)
                else:
                    stopW_1 = True
                
            elif ((stopW_0 == False) & (row[11] == 0)):
                countW_0 += 1
                if (countW_0 <= round(n1*ratio_w*(1-prob_w))):
                    cleanData.append(row)
                else:
                    stopW_0 = True        
    return cleanData

In [9]:
    RATIO_MEN=[0.5,0.65,0.8,0.95] 
    PROB_MEN=[0.05,0.20,0.35,0.5,0.65,0.8,0.95]
    DIFF_PROB=[-0.45,-0.30,-0.15,0,0.15,0.30,0.45]
    STD_ABILITY1=4
    STD_ABILITY2=3
    MEANS={'0.05':[20.88,10.88], '0.20':[22.9,12.9], '0.35':[24.035,14.035], '0.50':[25,15], '0.65':[25.965,15.965], '0.80':[27.1,17.1], '0.95':[29.12,19.12]}
    STD_NOISE=2.5
    NUMBER_DATASETS=550
    n=2000

    counter=0
    for ratio in RATIO_MEN:
        ratio=round(Decimal(ratio),2)
        for prob_men in PROB_MEN:
            prob_men=round(Decimal(prob_men),2)
            for diff_prob in DIFF_PROB:
                diff_prob=round(Decimal(diff_prob),2)
                if 0<prob_men+diff_prob<1:
                    prob_women=round(Decimal(prob_men+diff_prob),2)
                    data = []
                    for numb_dataset in range(1,NUMBER_DATASETS+1):
                        trainData =[]
                        testData =[]
                        seed=random.randint(0,10000)
                        random.seed(seed)
                        
                        for number in range(0,2):
                            if (number == 0):
                                for i in range(0,int(n*ratio)):
                                    trainData.append(createSample(counter+1,0))
                                for i in range(0,int(n*(1-ratio))):
                                    trainData.append(createSample(counter+1,1))
                                trainData = cleanDataSet(trainData, prob_men, ratio, diff_prob)
                                data.extend(trainData)
                            else:
                                for i in range(0,int(n*ratio)):
                                    testData.append(createSample(counter+1,0))
                                for i in range(0,int(n*(1-ratio))):
                                    testData.append(createSample(counter+1,1))
                                testData = cleanDataSet(testData, prob_men, ratio, diff_prob)
                                data.extend(testData)
                               
                                 
                    print('Exp. Group {}'.format(counter+1))
                    df = pd.DataFrame(data,columns=['Exp. Group', '# Dataset', 'Seed', 'Ratio Men', 'Probability Men', 'Probability Women', 'Difference Probabilities','Ability Score 1', 'Ability Score2', 'Height', 'Sex', 'Label'])
                    df.to_pickle('./Group{}'.format(counter+1))
                
                    counter+=1

Exp. Group 1
Exp. Group 2
Exp. Group 3
Exp. Group 4
Exp. Group 5
Exp. Group 6
Exp. Group 7
Exp. Group 8
Exp. Group 9
Exp. Group 10
Exp. Group 11
Exp. Group 12
Exp. Group 13
Exp. Group 14
Exp. Group 15
Exp. Group 16
Exp. Group 17
Exp. Group 18
Exp. Group 19
Exp. Group 20
Exp. Group 21
Exp. Group 22
Exp. Group 23
Exp. Group 24
Exp. Group 25
Exp. Group 26
Exp. Group 27
Exp. Group 28
Exp. Group 29
Exp. Group 30
Exp. Group 31
Exp. Group 32
Exp. Group 33
Exp. Group 34
Exp. Group 35
Exp. Group 36
Exp. Group 37
Exp. Group 38
Exp. Group 39
Exp. Group 40
Exp. Group 41
Exp. Group 42
Exp. Group 43
Exp. Group 44
Exp. Group 45
Exp. Group 46
Exp. Group 47
Exp. Group 48
Exp. Group 49
Exp. Group 50
Exp. Group 51
Exp. Group 52
Exp. Group 53
Exp. Group 54
Exp. Group 55
Exp. Group 56
Exp. Group 57
Exp. Group 58
Exp. Group 59
Exp. Group 60
Exp. Group 61
Exp. Group 62
Exp. Group 63
Exp. Group 64
Exp. Group 65
Exp. Group 66
Exp. Group 67
Exp. Group 68
Exp. Group 69
Exp. Group 70
Exp. Group 71
Exp. Group 72
E